In [88]:
import requests
import pandas as pd
import xlsxwriter
from datetime import datetime

In [89]:
#API URL
base_url = 'https://legislation.nysenate.gov/api/3/bills/search'

# API info
#api_key = "XLC9cIQDZOgmCiMCT43D1umf6VeOFuGU"


In [90]:
#adding a limit to increase it

def fetch_bill_details(api_key, session_year, page=1, term="", limit=25):
    """
    Fetch bill details from the NY Senate API.
    """
    offset = (page - 1) * limit  # Calculate the offset for pagination
    url = f"{base_url}?session_year={session_year}&key={api_key}&page={page}&term={term}&limit={limit}&offset={offset}"
    headers = {'Authorization': f'Token {api_key}'}
    
    try:
        response = requests.get(url, headers=headers)
        print(f"Fetching page {page}...")
        print(f"Response Status Code: {response.status_code}")
        
        # Raise an exception for any non-200 status codes
        response.raise_for_status()
        
        # Debugging: Print the full response structure if no error occurs
        print(f"Response Content: {response.text[:200]}...")  # Print only the first 200 chars to avoid long outputs
        
        return response.json()
    
    except requests.exceptions.RequestException as e:
        # If there was an error in the request, print the exception and response content if available
        print(f"Error getting data: {e}")
        if 'response' in locals():
            print(f"Response Content: {response.text}")  # Print the response body for more details
        return None

In [91]:
def flatten_bill_data(data, prefix=""):  
    """
    Flatten the JSON response from the API into a flat dictionary.
    """
    flattened = {}
    for key, value in data.items():
        new_key = f"{prefix}_{key}" if prefix else key
        if isinstance(value, dict):
            flattened.update(flatten_bill_data(value, new_key)) 
        elif isinstance(value, list):
            for i, item in enumerate(value):
                if isinstance(item, dict):
                    flattened.update(flatten_bill_data(item, f"{new_key}_{i}"))  # Flatten each item in the list
                else:
                    flattened[f"{new_key}_{i}"] = item
        else:
            flattened[new_key] = value
    return flattened


In [92]:
def export_to_excel(bill_data, filename_prefix="NY_Senate_Bills_data"):
    """
    Export the bill data to an Excel file.
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{filename_prefix}_{timestamp}.xlsx"

    if 'result' in bill_data and 'items' in bill_data['result']:
        items = bill_data['result']['items']
        flattened_items = []  # List to store flattened bill data

        for item in items:  # Flatten each bill item
            flattened_data = flatten_bill_data(item)
            flattened_items.append(flattened_data)

        df = pd.DataFrame(flattened_items)  # Create DataFrame from the list of flattened dictionaries

        if not df.empty:
            with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
                df.to_excel(writer, index=False, sheet_name='Bills')
                print(f"Data has been exported to {filename}")
        else:
            print("No data to export to Excel.")
    else:
        print("'result.items' not found in the API response or is empty.")

In [93]:
def fetch_all_bills(api_key, session_year, term="", limit=25):
    """
    Fetch all bills using pagination.
    """
    all_bills = []
    page = 1
    while True:
        bill_data = fetch_bill_details(api_key, session_year, page, term, limit)
        
        if bill_data and 'result' in bill_data and 'items' in bill_data['result']:
            items = bill_data['result']['items']
            if not items:
                break  # No more data to fetch
            all_bills.extend(items)
            page += 1  # Move to the next page
        else:
            print("No more data available or error encountered.")
            break

    return all_bills

In [94]:
# Main execution
session_year = 2022
api_key = "XLC9cIQDZOgmCiMCT43D1umf6VeOFuGU"

In [95]:
# Fetch all bills
all_bills = fetch_all_bills(api_key, session_year)

if all_bills:
    print(f"Fetched {len(all_bills)} bills in total.")
    export_to_excel({"result": {"items": all_bills}})  # Pass the list of bills directly for export
else:
    print("No data retrieved from the API.")

Fetching page 1...
Response Status Code: 200
Response Content: {
  "success" : true,
  "message" : "",
  "responseType" : "empty list",
  "total" : 0,
  "offsetStart" : 1,
  "offsetEnd" : 0,
  "limit" : 25,
  "result" : {
    "items" : [ ],
    "size" : 0
  }
}...
No data retrieved from the API.
